<a href="https://colab.research.google.com/github/hirenbioinfo/RNA_seq/blob/main/volcanoplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install rpy2==3.5.1
%load_ext rpy2.ipython

In [ ]:
%%R
library(tidyverse) # for data processing
library(RColorBrewer) # for a colourful plot
library(ggrepel) # for nice annotations
#install.packages("ggrepel") # if necessary


In [ ]:
%%R
# Import DGE results
df <- read.csv("/content/severevshealthy_degresults.csv", row.names = 1)
head(df)

In [ ]:
%%R
# Create a basic volcano plot
ggplot(data = df, aes(x = log2fc, y = -log10(pval))) +
  geom_point()

In [ ]:
%%R
# Add threshold lines
# Add threshold lines
ggplot(data = df, aes(x = log2fc, y = -log10(pval))) +
  geom_vline(xintercept = c(-0.6, 0.6), col = "red", linetype = 'dashed') +
  geom_hline(yintercept = -log10(0.05), col = "green", linetype = 'dashed') +
  geom_point()


In [ ]:
%%R
# Biostatsquid theme
theme_set(theme_classic(base_size = 20) +
            theme(
              axis.title.y = element_text(face = "bold", margin = margin(0,20,0,0), size = rel(1.1), color = 'black'),
              axis.title.x = element_text(hjust = 0.5, face = "bold", margin = margin(20,0,0,0), size = rel(1.1), color = 'black'),
              plot.title = element_text(hjust = 0.5)
            ))


In [ ]:
%%R
# Add a column to the data frame to specify if they are UP- or DOWN- regulated (log2fc respectively positive or negative)<br /><br /><br />
df$diffexpressed <- "NO"
df$diffexpressed[df$log2fc > 0.6 & df$pval < 0.05] <- "UP"
df$diffexpressed[df$log2fc < -0.6 & df$pval < 0.05] <- "DOWN"
head(df[order(df$padj) & df$diffexpressed == 'DOWN', ])

In [ ]:
%%R

# Create a new column "delabel" to de, that will contain the name of the top 30 differentially expressed genes (NA in case they are not)
df$delabel <- ifelse(df$gene_symbol %in% head(df[order(df$padj), "gene_symbol"], 30), df$gene_symbol, NA)
##############################################

ggplot(data = df, aes(x = log2fc, y = -log10(pval), col = diffexpressed)) +
  geom_vline(xintercept = c(-0.6, 0.6), col = "gray", linetype = 'dashed') +
  geom_hline(yintercept = -log10(0.05), col = "gray", linetype = 'dashed') +
  geom_point(size = 2) +
  scale_color_manual(values = c("#00AFBB", "grey", "#FFDB6D"),
                     labels = c("Downregulated", "Not significant", "Upregulated")) +
  ggtitle("Gene expression COVID vs healthy patients")




In [ ]:
%%R
ggplot(data = df, aes(x = log2fc, y = -log10(pval), col = diffexpressed, label = delabel)) +
  geom_vline(xintercept = c(-0.6, 0.6), col = "gray", linetype = 'dashed') +
  geom_hline(yintercept = -log10(0.05), col = "gray", linetype = 'dashed') +
  geom_point(size = 2) +
  scale_color_manual(values = c("#00AFBB", "grey", "#bb0c00"),
                     labels = c("Downregulated", "Not significant", "Upregulated")) +
  coord_cartesian(ylim = c(0, 250), xlim = c(-10, 10)) +
  labs(color = 'Severe', x = expression("log"[2]*"FC"), y = expression("-log"[10]*"p-value")) +
  scale_x_continuous(breaks = seq(-10, 10, 2)) +
  ggtitle('Thf-like cells in severe COVID-19 vs healthy patients') +
  geom_text_repel(max.overlaps = Inf)


In [ ]:
%%R
myvolcanoplot <- ggplot(data = df, aes(x = log2fc, y = -log10(pval), col = diffexpressed, label = delabel)) +
  geom_vline(xintercept = c(-0.6, 0.6), col = "gray", linetype = 'dashed') +
  geom_hline(yintercept = -log10(0.05), col = "gray", linetype = 'dashed') +
  geom_point(size = 2) +
  scale_color_manual(values = c("#00AFBB", "grey", "#bb0c00"),
                     labels = c("Downregulated", "Not significant", "Upregulated")) +
  coord_cartesian(ylim = c(0, 250), xlim = c(-10, 10)) +
  labs(color = 'Severe', x = expression("log"[2]*"FC"), y = expression("-log"[10]*"p-value")) +
  scale_x_continuous(breaks = seq(-10, 10, 2)) +
  ggtitle('Thf-like cells in severe COVID vs healthy patients') +
  geom_text_repel(max.overlaps = Inf)

# Open the file that will contain your plot (the name is up to you)
#pdf(file = "myvolcanoplot.pdf", width = 8, height = 12)  # You can change the size of the output file

# Execute the plot
#myvolcanoplot

# Close the file that contains the plot
#dev.off()

In [ ]:
%%R
myvolcanoplot